# Front End Lab 3: Aliasing and Anti-Aliasing Filter

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-


##Table of Contents

* [Introduction](#intro)
* [Task 0: Lab 2 Setup](#task0)
* [Task 1: Aliasing](#task1)
* [Task 2: Anti-Aliasing Filter](#task2)

<a id='intro'></a>
## Introduction

The block diagram below shows our circuit from last week. It amplifies the input voltage with a large gain and takes care of the voltage offset and voltage range of the ADC.

<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block-3.png">

As we have seen in lecture, it is very useful to see how our signal behaves in the frequency domain. We saw that when we sample a signal, if the input signal contains frequencies that are higher than half of the sampling frequency we will see aliasing. The high frequency component will show up as a lower frequency.

<a id='task0'></a>
##<span style="color:blue">Task 0: Lab 2 Setup</span>

Last week we modified our cascade of 3 amplifiers to include a DC Reject and a level-shift amplifier. Test each block and make sure they still work as expected before continuing with this lab. Prepare your function generator to output the BMI waveform with the following parameters:
- Sampling rate: <b>1 kSa/s</b>
- Amplitude: <b>100 mVpp</b>
- Offset: <b>0 V</b>

<a id='task1'></a>
##<span style="color:blue">Task 1: Aliasing</span>

###Materials
- Lab 2 setup

Let's take a look at the frequency content of our input signal. The data in the function generator was uploaded from the file `notMoving.csv`. Note that the plot below only shows the input signal - it does not show the noise added by the signal processing circuit.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = np.genfromtxt('notMoving.csv', delimiter=',')[:,1]
plt.plot(data)
plt.title('notMoving.csv')
plt.xlabel('Samples')
plt.ylabel('Magnitude')
data_fft = np.fft.rfft(data);
plt.figure()
plt.plot(np.linspace(0,500,len(data_fft)), abs(data_fft))
plt.title('FFT of notMoving.csv')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

**<span style="color:red">Given that the Launchpad samples at 333Hz, what is the desired bandwidth of our signal to avoid aliasing?</span>**

YOUR ANSWER HERE

**<span style="color:red">Looking at the FFT plot above, what is the most offending frequency component? What frequency would that frequency component be aliased to?</span>**

YOUR ANSWER HERE

Now let's take a look at the result of our ADC. Upload the `read_adc.ino` sketch to your Launchpad, connect pin `P6.0` to the output of the level-shift amplifier and the ground of the Launchpad to the ground of the circuit. Then run the cell below to gather 10 seconds of data. The code then takes the DFT of the collected data and plots the signal in the frequency domain. <b>It will take a while to run - be patient.</b>

In [ ]:
from adc_record import *
run('adc_before.txt')

In [ ]:
lines = list(open('adc_before.txt'))  
samples = [int(x)/4096 for x in lines]
samples = samples - np.mean(samples) # Normalizing to center at 0
plt.plot(samples)
plt.title('Recorded Signal')
plt.xlabel('Samples')
plt.ylabel('Magnitude')

In [ ]:
before_fft = np.fft.rfft(samples);

plt.plot(np.linspace(0,333/2,len(before_fft)), abs(before_fft))
plt.title('FFT of ADC output')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.xlim(0,333/2)

Yikes! That's very different from what we expect!

**<span style="color:red">What can we do to solve this issue? What kind of filter is needed here?</span>**

YOUR ANSWER HERE

<a id='task2'></a>
##<span style="color:blue">Task 2: Anti-Aliasing Filter</span>

###Materials
- Task 1 setup
- 1 10nF capacitor

As we have seen in lecture, we can remove both aliasing and high frequency noise using an anti-aliasing filter. One such filter is a low-pass filter. We are going to modify the first non-inverting amplifier we had after the DC reject circuit to an active low-pass filter circuit below.

<img style="width:350px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-lpf.png">

The circuit is called an active low-pass because it provides some gain and also attenuates high frequencies.

**<span style="color:red">Quickly derive the transfer function of the circuit above. Sketch out the magnitude of the transfer function with dB as y-axis and $log(\omega)$ as x-axis.</span>** If you're stuck, re-derive the transfer function.

**<span style="color:red">What is the gain of the amplifier at DC (0Hz)?</span>**

YOUR ANSWER HERE

**<span style="color:red">What is the 3dB cutoff frequency in terms of the components?</span>**

YOUR ANSWER HERE

**<span style="color:red">What is the 3dB cutoff frequency if $R_1 = 10k\Omega, R_2 = 100k\Omega$ and $C = 10nF$?</span>**

YOUR ANSWER HERE

Now modify your first non-inverting amplifier to the active low-pass circuit above. Your final circuit block diagram should look like below:
<img style="width: 750px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-block.png">

Before you continue, <b>there is a problem: the active low pass circuit has a finite input resistance!</b> This means it gives some loading to the stage right before it, the DC reject circuit in this case. If we leave it like this, the DC reject cutoff frequency will be shifted since the effective resistance of the filter is changed.

**<span style="color:red">What can we add between the DC Reject and the Active Low-Pass Filter to solve this issue?</span>** Hint: we have seen a simple circuit that isolates resistive loads. Check with a GSI or lab assistant before you continue.

YOUR ANSWER HERE

Now add the circuit you suggest above to solve the problem.

Probe both the function generator signal and the final output of the circuit - you will see that the amplitude gets reduced, but the signal is very clean compared to before.

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/fe-scope-LPF.png">

**<span style="color:red">Why is the output different from before? How does this filter effectively reduce aliasing?</span>**

YOUR ANSWER HERE

Now try running the cells below, which records 10 seconds of samples and plots the DFT of the signal just like before. Compare the DFT plot with the original signal.

In [ ]:
from adc_record import *
run('adc_after.txt')

In [ ]:
lines = list(open('adc_after.txt'))  
samples = [int(x)/4096 for x in lines]
samples = samples - np.mean(samples) # Normalizing to center at 0
plt.plot(samples)
plt.title('Recorded Signal')
plt.xlabel('Samples')
plt.ylabel('Magnitude')

In [ ]:
after_fft = np.fft.rfft(samples);

plt.plot(np.linspace(0,333/2,len(after_fft)), abs(after_fft))
plt.title('FFT of ADC output')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.xlim(0,333/2)

plt.figure()
plt.plot(np.linspace(0,500,len(data_fft)), abs(data_fft))
plt.xlim(0,333/2)
plt.title('FFT of notMoving.csv')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

How does it look? Satisfied?

Congratulations - you've completed the whole front-end chain that amplifies $\mu V$-amplitude signals to a range that is processable by the Launchpad ADC! You built a cascade of amplifiers and added both a DC reject and level shift, and solved the aliasing and high frequency noise problem with an active low-pass filter.

##Extra for experts

If you have some extra time, consider modifying the level-shift amplifier so it results in a larger signal amplitude. When we designed it before we assumed an input range of -200mV to 200mV, but if you probe the output of the low-pass filter now it actually has a smaller range because of some attenuation in the filter. Using the same circuit, choose new resistor values that brings the output to 0 - 3.3V so we get a larger dynamic range in the ADC readings.